In [1]:
import os
import time
import requests
import base64
from selenium.webdriver.common.by import By
import random
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.support import expected_conditions as EC

In [2]:
def create_folders(query):
    # Create the main "images" folder if it does not exis
    if not os.path.exists('datasetsingrid'):
        os.makedirs('datasetsingrid')

    # Create the training set folder within the "images" folder if it does not exist
    training_folder = os.path.join('datasetsingrid', 'training_set')
    if not os.path.exists(training_folder):
        os.makedirs(training_folder)

    # Create a folder within the training set folder with the query variable as its name if it does not exist
    query_training_folder = os.path.join(training_folder, query)
    if not os.path.exists(query_training_folder):
        os.makedirs(query_training_folder)

    # Create the test set folder within the "images" folder if it does not exist
    test_folder = os.path.join('datasetsingrid', 'test_set')
    if not os.path.exists(test_folder):
        os.makedirs(test_folder)

    # Create a folder within the test set folder with the query variable as its name if it does not exist
    query_test_folder = os.path.join(test_folder, query)
    if not os.path.exists(query_test_folder):
        os.makedirs(query_test_folder)

In [3]:
def download_images(image_urls, query, folder):
    #Loop through the retrieved urls.
    for i, image_url in enumerate(image_urls):
        #Fetch the content of the image url
        image_data = requests.get(image_url).content
        #If random.random is less than 0.8 the images is saved in the training set.
        #Otherwise they are saved in the test set, giving an approximate 80/20 split of the images.
        #This is not 100% exact.
        if random.random() <= 0.8:
            target_dir = os.path.join('datasetsingrid','training_set', folder)
        else:
            target_dir = os.path.join('datasetsingrid','test_set', folder)
        #Save the file in the correct folder
        with open(os.path.join(target_dir, f"{query}_{i}.jpg"), 'wb') as image_file:
                image_file.write(image_data)


In [4]:
def scrape_images(query):
    #Initialize the web driver/Open the browser
    driver = webdriver.Chrome()
    #The page to open with the query given as parameter
    driver.get(f"https://www.google.com/search?q={query}&tbm=isch")


    try:
        accept_all_button = driver.find_element(By.XPATH, "//button[contains(., 'Accept all')]")
        accept_all_button.click()
    except:
        pass



    #Scroll the page 10 times
    for _ in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
    #Initialize BeautifulSoup object    
    soup = BeautifulSoup(driver.page_source, 'lxml')
    #Close the driver/webbrowser
    driver.quit()
    #Find all the elements with the img tag and a specified class
    img_tags = soup.findAll('img', class_='rg_i Q4LuWd')
    #Empty list
    img_urls = []
    #Loop through the html tags
    for img_tag in img_tags:
        #Check for the data-src attribute
        if img_tag.has_attr('data-src'):
            #Extract the data-src value
            img_urls.append(img_tag['data-src'])
    #Return a list with the collected URLs
    return list(img_urls)

In [5]:
categories = {
    #'maple': ['maple tree leaf'],
    #'birch':['birch tree leaf','bjørkblad'],
    #'oak': ['oak tree leaf', 'oak leaf eikenblad'],
    #'fraxinus': ['fraxinus','ash three leaf'],
    #'salix_fragilis': ['salix_fragilis', 'crack willow tree leaf', 'brittle willow tree leaf']
    'ilex': ['ilex']
}


In [6]:
for key, values in categories.items():
    #Create folders with the key as folder name
    create_folders(key)
    #Loop over the values
    for value in values:
        #Scrape images with the values as search terms
        img_urls = scrape_images(value)
        #Download the scraped images
        download_images(img_urls,value, key)